In [1]:
%load_ext autoreload
%autoreload 2

import logging
from urllib.parse import urljoin
from io import StringIO
from typing import Sequence
from datetime import date, datetime

import requests
import pandas as pd
import numpy as np

from etf_scraper.base import SecurityListing
from etf_scraper.scrapers import ISharesListings, SSGAListings

logger = logging.getLogger(__name__)


ssga_listings = SSGAListings.retrieve_listings()        
ishares_listings = ISharesListings.retrieve_listings()

#TODO: Vanguard, Invesco - get proof of concept for one of ishares, ssga holdings first

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:286: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "inception_date"] = ssga_web_data_df_[
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:289: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "net_assets"] = ssga_web_data_df_["net_assets"].apply(
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:98: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set th

In [2]:
ssga_listings.tail()

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
205,The Industrial Select Sector SPDR® Fund,XLI,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.492249e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y704,US81369Y7040,Industrial Select Sector Index,SSGA
206,The Materials Select Sector SPDR® Fund,XLB,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,5.739410e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y100,US81369Y1001,Materials Select Sector Index,SSGA
207,The Real Estate Select Sector SPDR® Fund,XLRE,https://www.ssga.com/us/en/intermediary/etfs/f...,2015-10-07,US,5.051460e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y860,US81369Y8600,Real Estate Select Sector Index,SSGA
208,The Technology Select Sector SPDR® Fund,XLK,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,3.860053e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y803,US81369Y8030,Technology Select Sector Index,SSGA
209,The Utilities Select Sector SPDR® Fund,XLU,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.668948e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y886,US81369Y8865,Utilities Select Sector Index,SSGA


In [72]:
# ticker = 'SPY'.lower()
# url = (
#     "https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/"
#     f"etfs/us/holdings-daily-us-en-{ticker}.xlsx"
# )
# # ticker = 'SSFOX'.lower()
# # url = (
# #     "https://www.ssga.com/us/en/intermediary/ic/library-content/products/fund-data/"
# #     f"mf/us/holdings-monthly-us-en-{ticker}.xlsx"
# # )
# resp = requests.get(url)
# resp.raise_for_status()

# df = pd.read_excel(ef)

In [32]:
spy = SSGAListings.retrieve_holdings_("SPY")
spy

Ticker Symbol                  SPY
Holdings         As of 10-Jan-2023
nan                            NaN
Name: SPDR® S&P 500® ETF Trust, dtype: object
3


,name,ticker,cusip,sedol,weight,Sector,amount,local_currency,fund_ticker,as_of_date
4,Apple Inc.,AAPL,03783310,2046251,5.957167,Information Technology,166212290.000,USD,SPY,2023-01-10
5,Microsoft Corporation,MSFT,59491810,2588173,5.198587,Information Technology,82857736.000,USD,SPY,2023-01-10
6,Amazon.com Inc.,AMZN,02313510,2000019,2.430652,Consumer Discretionary,98652180.000,USD,SPY,2023-01-10
7,Berkshire Hathaway Inc. Class B,BRK.B,08467070,2073390,1.736894,Financials,20026460.000,USD,SPY,2023-01-10
8,Alphabet Inc. Class A,GOOGL,02079K30,BYVY8G0,1.609392,Communication Services,66391108.000,USD,SPY,2023-01-10
...,...,...,...,...,...,...,...,...,...,...
503,Lincoln National Corp,LNC,53418710,2516378,0.014801,Financials,1716026.000,USD,SPY,2023-01-10
504,DaVita Inc.,DVA,23918K10,2898087,0.013494,Health Care,616935.000,USD,SPY,2023-01-10
505,Fox Corporation Class B,FOX,35137L20,BJJMGY5,0.013357,Communication Services,1625095.000,USD,SPY,2023-01-10
506,DISH Network Corporation Class A,DISH,25470M10,2303581,0.011122,Communication Services,2735517.000,USD,SPY,2023-01-10


In [29]:
with open("test_SSGA_SPY_resp.xlsx", 'wb') as f:
    f.write(spy.content)

In [20]:
fhun = ishares_listings[ishares_listings['fund_name'].str.contains('500')].iloc[0]
fhun['ticker']

'IVV'

In [34]:
fhun_now = ISharesListings.retrieve_holdings_(fhun['ticker'], fhun['product_url'], None)
fhun_now

Index(['Ticker', 'Name', 'Sector', 'Asset Class', 'Market Value', 'Weight (%)',
       'Notional Value', 'Shares', 'Price', 'Location', 'Exchange', 'Currency',
       'FX Rate', 'Market Currency', 'Accrual Date'],
      dtype='object')


,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,AAPL,APPLE INC,Information Technology,Equity,1.789358e+10,6.00,1.789358e+10,134044353.0,133.49,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
1,MSFT,MICROSOFT CORP,Information Technology,Equity,1.575460e+10,5.28,1.575460e+10,66821890.0,235.77,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,7.565306e+09,2.53,7.565306e+09,79559430.0,95.09,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
3,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Financials,Equity,5.174177e+09,1.73,5.174177e+09,16150628.0,320.37,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,None
4,GOOGL,ALPHABET INC CLASS A,Communication,Equity,4.900160e+09,1.64,4.900160e+09,53541963.0,91.52,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,NWS,NEWS CORP CLASS B,Communication,Equity,2.093654e+07,0.01,2.093654e+07,1067102.0,19.62,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
505,ETD_USD,ETD USD BALANCE WITH R93531,Cash and/or Derivatives,Cash,7.500000e+01,0.00,7.500000e+01,75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,None
506,MARGIN_USD,FUTURES USD MARGIN BALANCE,Cash and/or Derivatives,Cash Collateral and Margins,-7.500000e+01,0.00,-7.500000e+01,-75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,None
507,ESH3,S&P500 EMINI MAR 23,Cash and/or Derivatives,Futures,0.000000e+00,0.00,6.629385e+08,3323.0,3990.00,NaN,Index And Options Market,USD,1.0,USD,NaN,IVV,None


In [30]:
fhun_ = ISharesListings.retrieve_holdings_(fhun['ticker'], fhun['product_url'], None) # date(2022, 12, 30)
fhun_

<Response [200]>

In [31]:
with open("./test_IShares_IVV_holdings_current_resp.csv", 'wb') as f:
    f.write(fhun_.content)

In [9]:
# # query_dates = pd.bdate_range('2022-01-01', '2023-01-01')

# dfs = {}

# for query_date in query_dates[:10]:
#     r = get_holdings(query_date)
#     dfs[query_date] = pd.read_csv(StringIO(r.content.decode()), skiprows=9, skipfooter=2)

In [289]:
from etf_scraper.scrapers import VanguardListings        

item_id = "0968"
holdings_resp = VanguardListings.retrieve_holdings_("VOO", item_id, date(year=2022, month=11, day=30))

{'as-of-date': '2022-11-30'}


/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:503: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  holdings_df.loc[:, "as_of_date"] = pd.to_datetime(holdings_df["as_of_date"])


In [290]:
holdings_resp

,as_of_date,ticker,sedol,cusip,name,amount,weight,security_type,sector,exchange,market_value,location,fund_ticker
0,2022-11-30,AAPL,2046251,037833100,Apple Inc.,3.461347e+08,6.468599,Equity,"Technology Hardware, Storage & Periphera",US,5.123832e+10,US,VOO
1,2022-11-30,MSFT,2588173,594918104,Microsoft Corp.,1.708825e+08,5.504166,Equity,Systems Software,US,4.359896e+10,US,VOO
2,2022-11-30,AMZN,2000019,023135106,Amazon.com Inc.,2.030818e+08,2.475106,Equity,Internet & Direct Marketing Retail,US,1.960552e+10,US,VOO
3,2022-11-30,GOOGL,BYVY8G0,02079K305,Alphabet Inc.,1.373847e+08,1.751589,Equity,Interactive Media & Services,US,1.387448e+10,US,VOO
4,2022-11-30,BRK.B,2073390,084670702,Berkshire Hathaway Inc.,4.135376e+07,1.663321,Equity,Multi-Sector Holdings,US,1.317531e+10,US,VOO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,2022-11-30,DVA,2898087,23918K108,DaVita Inc.,1.274545e+06,0.011864,Equity,Health Care Services,US,9.397220e+07,US,VOO
502,2022-11-30,VNO,2933632,929042109,Vornado Realty Trust,3.686253e+06,0.011769,Equity,Office REITs,US,9.322534e+07,US,VOO
503,2022-11-30,DISH,2303581,25470M109,DISH Network Corp.,5.745797e+06,0.011642,Equity,Cable & Satellite,US,9.222004e+07,US,VOO
504,2022-11-30,NaN,NaN,SLBBH1142,SLBBH1142,4.407205e+05,0.005563,Equity,NaN,US,4.406764e+07,NaN,VOO


In [251]:
van = VanguardListings.retrieve_listings()
van[van['ticker'].str.contains('VOO')]

,ticker,cusip,fund_name,inception_date,product_id,asset_class,subasset_class,fund_type,product_url,provider
32,VOO,922908363,Vanguard S&P 500 ETF,2010-09-07,0968,Stock Funds,Domestic Stock - General,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
33,VOOG,921932505,Vanguard S&P 500 Growth ETF,2010-09-07,3341,Stock Funds,Domestic Stock - General,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
34,VOOV,921932703,Vanguard S&P 500 Value ETF,2010-09-07,3340,Stock Funds,Domestic Stock - General,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
49,IVOO,921932885,Vanguard S&P Mid-Cap 400 ETF,2010-09-07,3342,Stock Funds,Domestic Stock - More Aggressive,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard


In [373]:
from etf_scraper.scrapers import InvescoListings

il = InvescoListings.retrieve_listings()
df = InvescoListings.retrieve_holdings_("QQQ")

In [375]:
with open('./Invesco_QQQ_holdings_20230113.csv', 'wb') as f:
    f.write(df)